## Model

In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import torch.utils.data
from skimage import io, transform
from skimage.measure import block_reduce
import numpy as np
import torch
import torch.utils.data
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models, transforms
import argparse

In [61]:
parser = argparse.ArgumentParser(description='Process display arguments')
parser.add_argument("-f", "--jupyter-json")
parser.add_argument('--batch-size', type=int, default=8, metavar='B', help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=8, metavar='TB', help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='E', help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--data_dir', type=str, default='../../processed_data/', metavar='F', help='Where to locate data')
parser.add_argument('--loss', type=str, default='l1', metavar='LOSS', help='l1, mse')
args = parser.parse_args()

In [70]:
class PairedTissueDataset(torch.utils.data.Dataset):
    def __init__(self, df_1, df_2, labels):
        self.df_1 = df_1
        self.df_2 = df_2
        self.labels = labels

    def __len__(self):
        return len(self.df_1)

    def __getitem__(self, idx):
        instance_name = self.df_1.iloc[idx].name
        label = int(self.labels.loc[instance_name])
        label = torch.Tensor([label])
        instance_1 = torch.Tensor(self.df_1.loc[instance_name])
        instance_2 = torch.Tensor(self.df_2.loc[instance_name])
        return (instance_1, instance_2, label)

labels = pd.read_csv("../../processed_data/labels.csv", index_col=0)
    
full_data = DataLoader(PairedTissueDataset(pd.read_csv("../../processed_data/full_muscle.csv", index_col=0), 
                                           pd.read_csv("../../processed_data/full_thyroid.csv", index_col=0), labels), 
                       batch_size=args.batch_size, shuffle=True) 

train_data = DataLoader(PairedTissueDataset(pd.read_csv("../../processed_data/train_muscle.csv", index_col=0), 
                                            pd.read_csv("../../processed_data/train_thyroid.csv", index_col=0), labels), 
                        batch_size=args.batch_size, shuffle=True) 

test_data = DataLoader(PairedTissueDataset(pd.read_csv("../../processed_data/test_muscle.csv", index_col=0), 
                                           pd.read_csv("../../processed_data/test_thyroid.csv", index_col=0), labels), 
                       batch_size=args.batch_size, shuffle=True) 


